In [ ]:
import os
import pandas as pd
import numpy as np

def merge_voltage_series(file_paths, output_path):
    dataframes = [pd.read_csv(fp, header=None) for fp in file_paths]
    n_rows = len(dataframes[0])

    # 첫 파일 마지막 시간값 → 반올림 → time_gap 계산
    last_time = dataframes[0].iloc[-1, 0]
    rounded_time = round(last_time)
    time_gap = rounded_time / n_rows
    print(f"자동 계산된 time_gap: {time_gap:.5f}")

    gap = int(0.2 * n_rows)

    # 최대 필요한 행 수 계산
    total_rows = 0
    for i in range(len(dataframes)):
        total_rows += n_rows
        if i != 0:
            total_rows += gap * i  # 누적 gap

    # 경고 출력
    if total_rows >= 1_000_000:
        print(f"⚠ 경고: 생성될 파일의 행 수가 {total_rows:,}개로 1,000,000을 초과합니다. 파일 크기에 주의하세요.")

    # 배열 초기화
    n_cols = len(dataframes) + 1  # time + 전압 열
    result_array = np.full((total_rows, n_cols), np.nan)

    # 시간열 생성
    result_array[:, 0] = np.arange(0, total_rows) * time_gap

    # 전압 데이터 삽입
    row_pos = 0
    for i, df in enumerate(dataframes):
        if i != 0:
            row_pos += gap * i
        result_array[row_pos:row_pos + n_rows, i + 1] = df.iloc[:, 1].values
        row_pos += n_rows

    # CSV 저장 (헤더 없이)
    pd.DataFrame(result_array).to_csv(output_path, index=False, header=False, float_format="%.6f")
    print(f"병합된 파일이 저장되었습니다: {output_path}")


In [ ]:
folder = r"C:/Users/limye/OneDrive - gachon.ac.kr/HI Lab/0. Projects/0. On going/2. Aloe inspired DEG/0. 실험자료/2. Collecting data/250624_12개의 샘플/minimized_data/depth"
file_names = ["D3.csv", "D4.csv", "D5.csv", "D6.csv"]
file_paths = [os.path.join(folder, f) for f in file_names]

output_path = os.path.join(folder, "Merge.csv")

merge_voltage_series(file_paths, output_path)

자동 계산된 time_gap: 0.000500
병합된 파일이 저장되었습니다: C:/Users/limye/OneDrive - gachon.ac.kr/HI Lab/0. Projects/0. On going/2. Aloe inspired DEG/0. 실험자료/2. Collecting data/250624_12개의 샘플/minimized_data/depth\Depth_generated.csv
